# Units

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: ../particles/index.rst
[plasmapy.formulary]: ../formulary/index.rst

This notebook introduces [astropy.units] with an emphasis on the functionality needed to work with [plasmapy.particles] and [plasmapy.formulary], as well as other PlasmaPy subpackages.

## Getting started with Astropy units

In scientific programming, it is common to represent physical quantities as numbers.

In [1]:
distance_in_miles = 50
time_in_hours = 2
velocity_in_mph = distance_in_miles / time_in_hours
print(velocity_in_mph)

25.0


[astropy.units]: https://docs.astropy.org/en/stable/units/index.html

Representing physical quantities as numbers has risks. We might accidentally perform operations with different units, like `time_in_seconds + time_in_hours`. We might even accidentally perform operations with physically incompatible units, like `length + time`.  Using [astropy.units] helps us avoid these problems. This subpackage is usually imported as `u`.

In [2]:
import astropy.units as u

We can create a physical quantity by multiplying a number with a unit.

In [2]:
distance = 80 * u.km
print(distance)

80.0 km


[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

This operation creates a [Quantity]: a number, sequence, or array that has been assigned a physical unit.

In [3]:
type(distance)

astropy.units.quantity.Quantity

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

Operations between [Quantity] objects handle the unit conversions automatically. We can add [Quantity] objects together as long as their units have the same physical type.

In [4]:
1 * u.m + 2 * u.cm

<Quantity 1.02 m>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can create [Quantity] objects with compound units.

In [5]:
time = 2.5 * u.hr
velocity = distance / time
print(velocity)

32.0 km / h


[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can create dimensionless [Quantity] objects too.

In [6]:
3 * u.dimensionless_unscaled

<Quantity 3.>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[.to]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to

The [.to] method allows us to convert a [Quantity] to different units of the same physical type. This method accepts strings that represent a unit (including compound units) or a unit object.

In [7]:
velocity.to("m/s")

<Quantity 8.88888889 m / s>

In [9]:
velocity.to(u.m / u.s)

<Quantity 11.11111111 m / s>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[.si]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.si
[.cgs]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.cgs

The [.si] and [.cgs] attributes convert the [Quantity] to SI or CGS units, respectively.

In [10]:
velocity.si

<Quantity 11.11111111 m / s>

In [11]:
velocity.cgs

<Quantity 1111.11111111 cm / s>

Attempting an operation between physically incompatible units gives us an error, which we can use to find bugs in our code.

In [12]:
3 * u.m + 3 * u.s

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

## Using Astropy units in PlasmaPy

[plasmapy.particles]: ../particles/index.rst
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[Particle]: ../api/plasmapy.particles.particle_class.Particle.rst

Now we can show some uses of Astropy units in PlasmaPy.  We can start with [plasmapy.particles].  Many of the attributes of the [Particle] class provide [Quantity] objects.

In [26]:
from plasmapy.particles import Particle

alpha = Particle("He-4 2+")

In [25]:
alpha.charge

<Quantity 3.20435327e-19 C>

In [24]:
alpha.mass

<Quantity 6.64465719e-27 kg>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[plasmapy.formulary]: ../formulary/index.rst

Similarly, [Quantity] objects are the expected inputs of most functions in [plasmapy.formulary]. Let calculate some plasma parameters for a typical region of the solar corona.

In [27]:
from plasmapy.formulary import Alfven_speed, gyrofrequency

In [44]:
B = 0.01 * u.T
n = 1e15 * u.m ** -3
ion = Particle("p+")

In [45]:
Alfven_speed(B=B, density=n, ion=ion).to("km /s")

<Quantity 6895.69343322 km / s>

## Equivalencies

Plasma scientists often use electron-volts (eV) as units of temperature, or more accurately, to describe the thermal energy per particle. However, we cannot convert eV to K because an eV is a unit of energy rather than temperature.

In [39]:
u.eV.to("K")

UnitConversionError: 'eV' (energy/torque/work) and 'K' (temperature) are not convertible

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[equivalencies]: https://docs.astropy.org/en/stable/units/equivalencies.html
[u.temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

To handle situations like this, [astropy.units] has built-in [equivalencies]. The conversion from eV to K can be done by using the [u.temperature_energy()] equivalency.

In [14]:
(1 * u.eV).to("K", equivalencies=u.temperature_energy())

<Quantity 11604.51812155 K>

## Using Astropy constants

[astropy.constants]: https://docs.astropy.org/en/stable/constants/index.html

We can use [astropy.constants] to access the most commonly needed physical constants.

In [15]:
from astropy.constants import c, e, k_B

print(c)

  Name   = Speed of light in vacuum
  Value  = 299792458.0
  Uncertainty  = 0.0
  Unit  = m / s
  Reference = CODATA 2018


[Constant]: https://docs.astropy.org/en/stable/api/astropy.constants.Constant.html#astropy.constants.Constant
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[u.temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

A [Constant] behaves very similarly to a [Quantity]. For example, we can use the Boltzmann constant to mimic the behavior of [u.temperature_energy()].

In [16]:
thermal_energy_per_particle = 0.6 * u.keV
temperature = thermal_energy_per_particle / k_B
print(temperature.to("MK"))

6.962710872930049 MK


Electromagnetic constants often need the unit system to be specified.

In [17]:
100 * e

TypeError: Constant 'e' does not have physically compatible units across all systems of units and cannot be combined with other values without specifying a system (eg. e.emu)

In [18]:
100 * e.si

<Quantity 1.60217663e-17 C>

## Physical types

[physical type]: https://docs.astropy.org/en/stable/units/physical_types.html
[physical_type]: https://docs.astropy.org/en/stable/api/astropy.units.UnitBase.html#astropy.units.UnitBase.physical_type
[get_physical_type()]: https://docs.astropy.org/en/stable/api/astropy.units.get_physical_type.html#astropy.units.get_physical_type

A [physical type] corresponds to physical quantities with dimensionally compatible units. Astropy has functionality that represents different physical types. These physical type objects can be accessed using either the [physical_type] attribute of units or [get_physical_type()].

In [47]:
(u.m ** 2 / u.s).physical_type

PhysicalType({'diffusivity', 'kinematic viscosity'})

In [49]:
u.get_physical_type("number density")

PhysicalType('number density')

Dimensional analysis can be done with these physical type objects.

In [51]:
energy_density = (u.J * u.m ** -3).physical_type
velocity = u.get_physical_type("velocity")
energy_density * velocity

PhysicalType({'energy flux', 'irradiance'})